In [1]:
import numpy as np
import numba as nb

In [9]:
# CONSTANTS

L = 7
W = (L+1)//2
INITIAL_SAT = 1

# to compare with cartesian, origin in upper right corner and e_1 = [1,0], e_2 = [0,-1]
# REL_NEIGHBOR_POS_LIST = [
#     (0,-1),
#     (1,-1),
#     (1,0),
#     (0,1),
#     (-1,1),
#     (-1,0)
# ]

# initialisation



@nb.njit
def initialize_sat_map(l, w, initial_sat):
    sat_map = np.zeros((l,w), dtype=np.float64)

    for line in range(l):
        for col in range((l - line + 1)//2):
            sat_map[line, col] = initial_sat


def construct_minimal_ice_map(l, w):
    ice_map = np.full((l, w), False, dtype=bool)

    # minimum amount of initial ice that doesn't brick the model
    minimal_ice_coords = [
        (l-1,0),
        (l-2,0),
        (l-3,0),
        (l-3,1)
    ]

    for tup in minimal_ice_coords:
        ice_map[tup[0], tup[1]] = True

    return ice_map


@nb.njit
def is_legit_cell(line, col, L):
    return line<L and col<(L - line + 1)//2 and line>=0 and col>=0


@nb.njit
def get_neighbors(line, col, L):
    relative_nearest_neighbors = [
        (-1,0),
        (-1,1),
        (0,1),
        (1,0),
        (1,-1),
        (0,-1)
    ]

    absolute_nearest_neighbors = np.empty((6, 2))

    for i in range(len(relative_nearest_neighbors)):
        neighbor_line = line + relative_nearest_neighbors[i][0]
        neighbor_col = col + relative_nearest_neighbors[i][1]

        if is_legit_cell(neighbor_line, neighbor_col, L):
            absolute_nearest_neighbors[i, :] = [neighbor_line, neighbor_col]
        else:
            absolute_nearest_neighbors[i, :] = [np.nan, np.nan]

    return absolute_nearest_neighbors


@nb.njit
def construct_neighbor_array(l, w):
    neighbor_array = np.empty((l, w, 6, 2))

    for line in range(l):
        for col in range((l - line + 1)//2):
            neighbor_array[line, col, :, :] = get_neighbors(line, col, l)

    return neighbor_array


@nb.njit
def construct_boundary_map(ice_map, neighbor_array):
    l = np.shape(ice_map)[0]
    boundary_map = np.full_like(ice_map, 0, dtype=np.int8)

    for line in range(l):
        for col in range((l - line + 1)//2):
            neighbor_coords = neighbor_array[line, col, :, :] # returns 6x2 array
            
            neighbor_counter = 0

            if ice_map[line, col] != True:
                for i in range(np.shape(neighbor_coords)[0]):
                    neighbor_line = int(neighbor_coords[i,0])
                    neighbor_col = int(neighbor_coords[i,1])

                    if not np.isnan(neighbor_line):
                        if ice_map[neighbor_line, neighbor_col] == True:
                            neighbor_counter += 1

            boundary_map[line, col] = neighbor_counter

    return boundary_map
                

def construct_default_diffusion_rules(l, w):
    diffusion_rules = np.empty((l, w), dtype=list)

    for line in range(1, l-3):
        diffusion_rules[line, 0] = [
            1/6,
            1/3,
            1/3,
            1/6,
            0, 
            0
        ]



In [10]:
test_l = 5
test_w = 3

test_neighbor_array = construct_neighbor_array(test_l, test_w)
test_ice_map = construct_minimal_ice_map(test_l, test_w)
test_boundary_map = construct_boundary_map(test_ice_map, test_neighbor_array)

# get_neighbors(0,0,1001)
# NA = construct_neighbor_array(101, 51)


In [11]:
print(test_ice_map)
print(test_boundary_map)

[[False False False]
 [False False False]
 [ True  True False]
 [ True False False]
 [ True False False]]
[[0 0 0]
 [1 2 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]
